<h1 style="color:#c6b080; font-size:30px; text-align:center; background-color:#cefffc; padding:20px; border-radius:10px;">Projet bloc 1 : moteur de recommendations de livres</h1>

<h1>Script de l'extraction automatisée des données</h1>

In [ ]:
from datetime import datetime
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import sys
import xml.etree.ElementTree as ET
from deep_translator import GoogleTranslator
from dotenv import load_dotenv
import os
import base64
from tqdm import tqdm

start_time = time.time()  

load_dotenv()

token_librarything_API = os.getenv('TOKEN_LIBRARYTHING_API')

print("\nDébut de l'exécution du script :",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Environnement virtuel requis : env_wsl_bloc_1.")
print("Environnement virtuel et Python utilisés =", sys.executable)

############################################################## Définition des fonctions d'extraction de données
def is_french_isbn(isbn):
    """Détermine si l'ISBN est français ou pas selon selon la détection des préfixes :
    "9782" ou "97910" pour les ISBN13
    "2" pour les ISBN10."""
    isbn = isbn.replace("-", "").replace(" ", "")
    if isbn.startswith("9782") or isbn.startswith("97910") or isbn.startswith("2"):
            return True
    return False

def find_french_isbn_in_list(isbns):
    """Parcourt une liste d'ISBN jusqu'à en trouver un qui soit français, puis le retourne.
    Détermine si l'ISBN est français ou pas selon selon la détection des préfixes :
    "9782" ou "97910" pour les ISBN13
    "2" pour les ISBN10.  """
    count = 0
    while count < len(isbns):
        isbn = isbns[count]
        isbn = isbn.replace("-", "").replace(" ", "")
        if isbn.startswith("9782") or isbn.startswith("97910") or isbn.startswith("2"):
            return isbn
        count += 1
    return None

def get_isbn_with_librarything(token_librarything_API,title):
    """Retourne une liste de tous les ISBN connus par l'API librarything pour ce livre à partir du titre donné.
    Ecrivez le titre naturellement en conservant les espaces."""
    title = title.replace(' ', '+')
    url = f"https://www.librarything.com/api/{token_librarything_API}/thingTitle/{title}"
    response = requests.get(url)
    if response.status_code == 200:
        root = ET.fromstring(response.content.decode('utf-8'))
        liste = []
        for elem in root.iter("isbn"):
            liste.append(elem.text)
        return liste
    else :
        return None

def get_book_from_isbn(isbn):
    """lance un requests.get(url) vers une page de librarything depuis l'ISBN"""
    url = f"https://www.librarything.com/isbn/{isbn}"
    response = requests.get(url)
    if response.status_code == 200:
        print("requete reussie, pret a scraper")
    else:
        print(f"Erreur lors de la requête https://www.librarything.com/isbn/ : {response.status_code}")

def translate_title_to_french(titre):
    traduction = GoogleTranslator(source='auto', target='fr').translate(titre)
    return traduction

def google_books_api(title, maxresults = 15):
    """interroge google_books_api avec le titre d'un livre. 
    Retourne un tuple data, title, page_count, isbn_fr, author, theme, response.status_code.
    Argument 1 = titre du livre (on peut laisser les espaces)
    Argument 2 = nombre de résultats maximal pour la requête"""
    url = "https://www.googleapis.com/books/v1/volumes"
    params = {
        "q": f"intitle:{title}",
        "maxResults": maxresults
    }
    response = requests.get(url, params=params)
    data = response.json()
    if "items" in data:
        ######################################################### Extraire le titre
        title = data["items"][0]["volumeInfo"]["title"]
        if not title :
            title = None
        ######################################################### Extraire le nombre de pages
        page_count = 0
        if data.get("items"):
            for livre in data["items"]:
                if livre.get("volumeInfo"):
                    volumeInfo = livre.get("volumeInfo")
                    if volumeInfo.get("pageCount"):
                        if volumeInfo.get("pageCount") > 5:
                            page_count = volumeInfo.get("pageCount")
        if not page_count:
            page_count = None
        ######################################################### Extraire un ISBN français
        isbn_fr = ""
        if data.get("items"):
            for livre in data["items"]:
                if livre.get("volumeInfo"):
                    volumeInfo = livre.get("volumeInfo")
                    if volumeInfo.get("industryIdentifiers"):
                        industryIdentifiers = volumeInfo.get("industryIdentifiers")
                        for i in industryIdentifiers :
                            if i.get("identifier"):
                                code = i.get("identifier")
                                if len(code) == 10 and code.startswith("2"):
                                    isbn_fr = code 
                                if code.startswith("9782") or code.startswith("97910"):
                                    isbn_fr = code
        if not isbn_fr:
            isbn_fr = None   
        ######################################################### Extraire l'auteur
        author = ""
        if data.get("items"):
            for livre in data["items"]:
                if livre.get("volumeInfo"):
                    volumeInfo = livre.get("volumeInfo")
                    if volumeInfo.get("authors"):
                        author = volumeInfo.get("authors")
        if not author:
            author = None

        ######################################################### Extraire les thèmes
        theme = ""
        if data.get("items"):
            for livre in data["items"]:
                if livre.get("volumeInfo"):
                    volumeInfo = livre.get("volumeInfo")
                    if volumeInfo.get("categories"):
                        theme = str(volumeInfo.get("categories"))
        if not theme:
            theme = None
        
        return data, title, page_count, isbn_fr, author, theme, response.status_code
    else : 
        return None, None, None, None, None, None, response.status_code

########################################################################################## fin de la définition des fonctions
########################################################################################## début du scraping
# print("\nDébut du webscraping de Openlibrary.org")

titres = []
notes = []
auteurs = []
dates_de_publication = []
editeurs = []
nombres_de_pages = []
themes_liste = []
isbn10_français = []
isbn13_français = []
isbn_defaut = []
ISBN_verifie = []
images_openlibrary = []

for i in tqdm(range(1,2), desc="Scraping des 10 pages \"les livres en vogue du moment\" de Openlibrary.org"):
    url_scrap = f"https://openlibrary.org/trending/now?page={i}"
    response = requests.get(url_scrap)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        liens = soup.find_all("a", class_="results")
        # print("\n\n\nextraction des liens et redirection vers les livres de la page",i)
        for title in liens:
            url2 = f"""https://openlibrary.org/{title.get("href")}"""
            response2 = requests.get(url2)
            if response2.status_code == 200:
                soup2 = BeautifulSoup(response2.text, "html.parser")
                
                try:
                    titre = soup2.find("h1", itemprop="name").text
                    titres.append(titre)
                    # print("\nExtraction des informations du livre",titre)
                except Exception as e:
                    # print("Echec de récupération du titre dans\n\t\t\t",url2,"\n\t\t\t",e,"\n")
                    titres.append("indisponible")
                    titre = "indisponible"
                
                #exctraction des ISBN 10
                try:
                    isbn = soup2.find_all("dd", attrs={"itemprop": "isbn"})
                    french_isbn = ""
                    for number in isbn:
                        s = number.text
                        s = re.sub(r"[^A-Z0-9]", "", s)
                        if len(s) == 10: 
                            if is_french_isbn(s):
                                french_isbn = s
                            else:
                                def_isbn = s
                    if french_isbn:
                        value10 = french_isbn
                    else :
                        value10 = "indisponible"
                except Exception as e:
                    value10 = "indisponible"
                    
                isbn10_français.append(value10)

               #exctraction des ISBN 13
                try:
                    isbn = soup2.find_all("dd", attrs={"itemprop": "isbn"})
                    french_isbn = ""
                    def_isbn =""
                    for number in isbn:
                        s = number.text
                        s = re.sub(r"[^A-Z0-9]", "", s)
                        if len(s) == 13: 
                            if is_french_isbn(s):
                                french_isbn = s
                            else:
                                def_isbn = s
                    if french_isbn:
                        value13 = french_isbn
                    else :
                        value13 = "indisponible"
                except Exception as e:
                    value13 = "indisponible"
                    
                isbn13_français.append(value13)

                if def_isbn:
                    isbn_defaut.append(def_isbn)
                else:
                    def_isbn = "indisponible"
                    isbn_defaut.append(def_isbn)
                    
                if value10 == "indisponible" and value13 == "indisponible":
                    # print("""\tAucun ISBN français trouvé pendant le scraping d'Openlibrary.org.\n\t\tRequête à l'API Librarything en cours pour tenter de récupérer l'ISBN français.""")
                    res = get_isbn_with_librarything(token_librarything_API,titre)
                    if res:
                        # print("\t\tLibrarything propose",len(res),"ISBN...")
                        value = find_french_isbn_in_list(res)
                        if value:
                            if len(value) == 10:
                                value10 = value
                                isbn10_français.pop()
                                isbn10_français.append(value)
                                # print("\t\t...un ISBN 10 français a pu être extrait :",value)
                            if len(value) == 13:
                                value13 = value
                                isbn13_français.pop()
                                isbn13_français.append(value)
                                # print("\t\t... un ISBN 13 français a pu être extrait :",value)
                        if not value :
                            # print("\t\t... mais aucun n'est français.\n\t\tNouvelle tentative en cours vers l'API google books.")
                            a = google_books_api(titre)
                            if a[3]:
                                # print("\t\tRésultat avec google books =",a[3])
                                if a[3] == 10:
                                    value10 = a[3]
                                    isbn10_français.pop()
                                    isbn10_français.append(value10)
                                else:
                                    value13 = a[3]
                                    isbn13_français.pop()
                                    isbn13_français.append(value13)
                            else :
                                # print("\t\tGoogle books n'a pas trouvé d'ISBN français pour", titre)
                                # print("\t\tNouvelle tentative avec cette traduction du titre :", translate_title_to_french(titre))
                                res_translated = google_books_api(translate_title_to_french(titre),3)[3]
                                if res_translated:
                                    # print("\t\tAvec ce nouveau titre, Google books API a trouvé cet ISBN français:",res_translated)
                                    if res_translated == 10:
                                        value10 = res_translated
                                        isbn10_français.pop()
                                        isbn10_français.append(value10)
                                    else:
                                        value13 = res_translated
                                        isbn13_français.pop()
                                        isbn13_français.append(value13)
                                # else :
                                    # print("\t\tLa traduction du titre n'a pas donné de meilleurs résultats.")
                    else:
                        # print("\t\tAucun ISBN trouvé avec l'API de librarything.\n\t\tNouvelle tentative en cours vers l'API google books.")
                        a = google_books_api(titre)
                        if a[3]:
                            # print("\t\tRésultat avec google books =",a[3])
                            if a[3] == 10:
                                value10 = a[3]
                                isbn10_français.pop()
                                isbn10_français.append(value10)
                            else:
                                value13 = a[3]
                                isbn13_français.pop()
                                isbn13_français.append(value13)
                        else :
                            # print("\t\tGoogle books n'a pas trouvé d'ISBN français pour", titre)
                            # print("\t\tNouvelle tentative avec cette traduction du titre :", translate_title_to_french(titre))
                            a = google_books_api(translate_title_to_french(titre),3)[3]
                            if a:
                                # print("\t\tAvec ce nouveau titre, Google books API a trouvé cet ISBN français:",a)
                                if a == 10 :
                                    isbn10_français.pop(0)
                                    isbn10_français.append(a)
                                    value10 = a
                                else:
                                    isbn13_français.pop()
                                    isbn13_français.append(a)
                                    value13 = a
                            # else :
                                # print("\t\tLa traduction du titre n'a pas donné de meilleurs résultats.")
                                
                # if value10 == "indisponible" and value13 == "indisponible" and def_isbn == "indisponible":
                    # print("\t\tAucun isbn extrait.")

                ISBN_verifie.append("ISBN pas encore vérifié")
                #exctraction des notes    
                try:
                    note = soup2.find("span", itemprop="ratingValue").text
                    # note = re.sub(r"\(.*?\)", "", note)
                    notes.append(note)
                except Exception as e:
                    # print("\tEchec de récupération de la note.")
                    notes.append("indisponible")

                #extraction des auteurs
                try:
                    auteur = soup2.find("a", itemprop="author").text
                    if auteur :
                        auteurs.append(auteur)
                    else :
                        # print("\tPas d'auteur extrait depuis Openlibrary.")
                        a = str(google_books_api(titre)[4])
                        if a:
                            auteurs.append(a)
                            # print("\t\tAuteur récuépré via Google_books_API :",a)
                        else:
                            # print("\t\tEchec de récupération de l'auteur, même après une deuxième tentative sur google_books_API.")
                            auteurs.append("indisponible")
                except Exception as e:
                    # print("\tEchec de récupération de l'auteur.")
                    auteurs.append("indisponible")

                #extraction des dates de publication
                try:
                    date_de_publication = soup2.find("span", itemprop="datePublished").text
                    dates_de_publication.append(date_de_publication)
                except Exception as e:
                    # print("\tEchec de récupération de la date de publication.")
                    dates_de_publication.append("indisponible")

                #extraction des éditeurs
                try:
                    editeur = soup2.find("a", itemprop="publisher").text
                    editeurs.append(editeur)
                except Exception as e:
                    # print("\tEchec de récupération de l'éditeur.")
                    editeurs.append("indisponible")

                #extraction des nombres de pages
                try:
                    nombre_de_pages = soup2.find("span", itemprop="numberOfPages").text
                    if nombre_de_pages:
                        if int(nombre_de_pages) > 5: 
                            nombres_de_pages.append(int(nombre_de_pages))
                        else:
                            nombres_de_pages.append("indisponible")
                    else :
                        # print("\tPas de nombre de pages extrait depuis Openlibrary.")
                        a = google_books_api(titre)[2]
                        if a and int(a) > 5:
                            # print("\t\tNombre de pages récuépré via Google_books_API :",a)
                            nombres_de_pages.append(int(a))
                        else:
                            # print("\t\tEchec de récupération du nombre de pages, même après une deuxième tentative sur google_books_API.")
                            nombres_de_pages.append("indisponible")
                except Exception as e:
                    # print("\tEchec de récupération du nombre de pages.")
                    a = google_books_api(titre)[2]
                    if a and int(a) > 5:
                        # print("\t\tNombre de pages récupéré via Google_books_API :",a)
                        nombres_de_pages.append(int(a))
                    else:
                        # print("\t\tEchec de récupération du nombre de pages, même après une deuxième tentative sur google_books_API.")
                        nombres_de_pages.append("indisponible")
        
                #extraction des thèmes
                try:
                    themes_tags = soup2.find_all("a", attrs={"data-ol-link-track": "BookOverview|SubjectClick"})
                    themes_sous_liste = []
                    for tag in themes_tags :
                        themes_sous_liste.append(tag.text)
                    if themes_sous_liste:
                        themes_liste.append(themes_sous_liste)
                    else:
                        # print("\tEchec de récupération des thèmes")
                        theme = google_books_api(titre)[5]
                        if theme :
                            themes_liste.append(theme)
                            # print("\t\tThèmes récupérés via Google_books_API")
                        else:
                            themes_liste.append("indisponible")
                            # print("\t\tEchec également avec Google_books_API")
                except Exception as e:
                    # print("\tEchec de récupération des thèmes")
                    theme = google_books_api(titre)[5]
                    if theme :
                        themes_liste.append(theme)
                        # print("\t\tThèmes récupérés via Google_books_API")
                    else:
                        themes_liste.append("indisponible")
                        # print("\t\tEchec également avec Google_books_API")

                #récupération des images
                try:
                    boite_image = soup2.find("div", class_="illustration edition-cover")
                    img_tag = boite_image.find("img")
                    img_url = "https:" + img_tag["src"]
                    response_img = requests.get(img_url)
                    if response_img.status_code == 200 :
                        image_64 = base64.b64encode(response_img.content).decode('utf-8')
                        if image_64:
                            images_openlibrary.append(image_64)          
                        else:
                            images_openlibrary.append("indisponible")
                    else :
                        images_openlibrary.append("indisponible")
                except Exception as e:
                    # print("\tEchec de récupération de l'image dans\n\t\t",url2,"\n\t\t",e,"\n")
                    images_openlibrary.append("indisponible")
            else:
                print("\t!!redirection vers la description du livre ratée, erreur =",response2.status_code)
    else:
        print("\t!!", datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "echec de la requete ",url_scrap, " status_code erreur =",response.status_code)



themes = []
for i in themes_liste:
    themes.append(str(i))    

data_webscrap_a_nettoyer = {
    "livre_nom" : titres,
    "livre_note" : notes,
    "livre_nombre_de_pages" : nombres_de_pages,
    "livre_date" : dates_de_publication,
    "ISBN_10" : isbn10_français,
    "ISBN_13" : isbn13_français,
    "ISBN_defaut" : isbn_defaut,
    "ISBN_verifie" : ISBN_verifie,
    "thematique_nom" : themes,
    "editeur_nom" : editeurs,
    "auteur_nom" : auteurs,
    "images_openlibrary" : images_openlibrary
    }

print(len(titres),"titres")
print(len(notes),"notes")
print(len(nombres_de_pages),"nombres de pages")
print(len(dates_de_publication),"dates")
print(len(isbn10_français),"isbn 10")
print(len(isbn13_français),"isbn 13")
print(len(isbn_defaut),"isbn_defaut")
print(len(themes),"thèmes")
print(len(editeurs),"editeurs")
print(len(auteurs),"auteurs")
print(len(images_openlibrary),"images_openlibrary")

df_data_webscrap_a_nettoyer = pd.DataFrame(data_webscrap_a_nettoyer)

missing_livres = df_data_webscrap_a_nettoyer['livre_nom'].value_counts().get('indisponible', 0)
missing_notes = df_data_webscrap_a_nettoyer['livre_note'].value_counts().get('indisponible', 0)
missing_auteurs = df_data_webscrap_a_nettoyer['auteur_nom'].value_counts().get('indisponible', 0)
missing_dates = df_data_webscrap_a_nettoyer['livre_date'].value_counts().get('indisponible', 0)
missing_editeurs = df_data_webscrap_a_nettoyer['editeur_nom'].value_counts().get('indisponible', 0)
missing_pages = df_data_webscrap_a_nettoyer['livre_nombre_de_pages'].value_counts().get('indisponible', 0)
missing_themes = df_data_webscrap_a_nettoyer['thematique_nom'].value_counts().get('indisponible', 0)
missing_isbn = ((df_data_webscrap_a_nettoyer["ISBN_10"] == "indisponible") & (df_data_webscrap_a_nettoyer["ISBN_13"] == "indisponible")& (df_data_webscrap_a_nettoyer["ISBN_defaut"] == "indisponible")).sum()
missing_images = df_data_webscrap_a_nettoyer['images_openlibrary'].value_counts().get('indisponible', 0)
total_missing = int(missing_images)+int(missing_livres)+int(missing_notes)+int(missing_auteurs)+int(missing_dates)+int(missing_editeurs)+int(missing_pages)+int(missing_themes)+int(missing_isbn)

print("\nRapport de Webscraping : sur",len(df_data_webscrap_a_nettoyer), " livres:\n" )
print("\t\tTitres indisponibles =", missing_livres)
print("\t\tNotes indisponibles =", missing_notes)
print("\t\tAuteurs indisponibles =", missing_auteurs)
print("\t\tDates_de_publication indisponibles =", missing_dates)
print("\t\tEditeurs indisponibles =", missing_editeurs)
print("\t\tNombres de pages indisponibles =", missing_pages)
print("\t\tThèmes indisponibles =", missing_themes)
print("\t\tLivres sans aucun ISBN =", missing_isbn)
print("\t\tLivres sans image =", missing_images)

objectif = len(df_data_webscrap_a_nettoyer)*9
reussite = objectif - total_missing
success_rate = reussite / objectif*100
print("\n\t Taux de réussite des extractions =", success_rate,"%")
if success_rate >= 92:
    print("\n\t Excellent travail, Johnson.\n")
elif success_rate >= 86:
    print("\n\t Pas mal, Johnson.\n")
elif success_rate >= 78:
    print("\n\t C'est presque mauvais, Johnson.\n")
else :
    print("\n\t Je ne vous paye pas pour des résultats aussi mauvais, Johnson.\n")


#csv pour nettoyage automatisé
try:
    df_data_webscrap_a_nettoyer.to_csv('df_data_webscrap_a_nettoyer.csv', index=False)
    print('Le fichier "df_data_webscrap_a_nettoyer.csv" a bien été généré.')
except Exception as e:
    print(f"Erreur lors de la création du fichier CSV : {e}")


#csv pour archive
try:
    a = {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
    a = re.sub("['{}]","",re.sub("[-: ]","_",str(a)))
    df_data_webscrap_a_nettoyer.to_csv(f'archives_csv/df_data_webscrap_a_nettoyer_{a}.csv', index=False)
    print('Le fichier csv d\'archive a bien été généré.')
except Exception as e:
    print(f"Erreur lors de la création du fichier d\'archive : {e}")

end_time = time.time()  
execution_time = (end_time - start_time)/60
print(f"Temps d'exécution : {execution_time:.4f} minutes")

<h1>Script d'insertion en BDD MySql - Mongo </h1>

In [ ]:
# CREER LA BASE DE DONNEES DANS L'INVITE DE COMMANDE :
# Se mettre dans le dossier qui contient fichier.sql puis écrire la commande :
# "C:\Program Files\MySQL\MySQL Server 8.0\bin\mysql.exe" < bdd.sql -u root -p
from datetime import datetime
import mysql.connector
import os
from dotenv import load_dotenv
import csv
import pandas as pd
import numpy as np
import pymongo
from pymongo import MongoClient
print("\nExécution du script d'insertion:",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
load_dotenv()

################################################################### Partie MySQL
user = os.getenv('USER')
password = os.getenv('PASSWORD')
host = os.getenv('HOST')
database = os.getenv('DATABASE')

connexion = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)
curseur = connexion.cursor()
# Le curseur est utilisé pour exécuter des requêtes SQL
# curseur.execute("select * from livres;")
# for i in curseur:
#      print(i[0])

    
def remplir_sql(table : str, csv : str, connexion):
    """Fonction qui prend en argument la table à remplir en str, le csv source en str, et la connexion mysql.connector.
    Le passage en csv fait perdre les d-types des colonnes pandas. Ici, ils seront retypés avec astype() pour permettre une transformation des 
    NaN en None afins que ces valeurs manquantes soient admises par MySQL"""
    df = pd.read_csv(csv)
    if csv == "table_livres.csv":
        df["ISBN_defaut"] = df["ISBN_defaut"].astype("str") #df["ISBN_defaut"] ressort des floats. Il faut traiter ça
        def modify_isbn(value):
            try:
                return value[:-2] if float(value) > 5 else value
            except ValueError:
                return value
        df["ISBN_defaut"] = df["ISBN_defaut"].apply(modify_isbn)
        df["ISBN_defaut"] = df["ISBN_defaut"].replace("nan", None)
        df["ISBN_10"] = df["ISBN_10"].astype("str")
        df["ISBN_10"] = df["ISBN_10"].apply(modify_isbn)
        df["ISBN_10"] = df["ISBN_10"].replace("nan", None)
        df["livre_nombre_de_pages"] = df["livre_nombre_de_pages"].astype("object")
        df["livre_note"] = df["livre_note"].astype("object")
        df["ISBN_verifie"] = df["ISBN_verifie"].astype(bool)
        df["livre_date"] = df["livre_date"].astype("object")
        df = df.where(pd.notna(df), None) # where(condition, autre_valeur) conserve les valeurs condition == True sinon remplace par autre_valeur
    else:
        df = df.where(pd.notna(df), None) 
        
    # Récupération des colonnes du dataframe au format string pour définir les colonnes cibles dans la requête sql
    colonnes = ', '.join(df.columns)
    # On génère le bon nombre de placeholders (les %s) au format string pour définir les valeurs à insérer dans la requête sql
    placeholders = ', '.join(['%s'] * len(df.columns))
    sql = f"INSERT INTO {table} ({colonnes}) VALUES ({placeholders})"
    # Définition des valeurs avec des tuples :
    # df.itertuples() est une méthode qui parcourt un DataFrame et extrait les lignes sous forme de tuples, 
    # ce qui est parfait pour executemany()
    valeurs = [tuple(row) for row in df.itertuples(index=False, name=None)]
    curseur = connexion.cursor()
    curseur.executemany(sql, valeurs)
    connexion.commit()
    curseur.close()
    print("Inserts réussis dans",table)

remplir_sql("Livres", "table_livres.csv", connexion)
remplir_sql("Auteurs", "table_auteurs.csv", connexion)
remplir_sql("Editeurs", "table_editeurs.csv", connexion)
remplir_sql("Thematiques", "table_thematiques.csv", connexion)
remplir_sql("auteurs_livres", "table_auteurs_livres.csv", connexion)
remplir_sql("editeurs_livres", "table_editeurs_livres.csv", connexion)
remplir_sql("thematiques_livres", "table_thematiques_livres.csv", connexion)

connexion.close()

################################################################### Partie MongoDB
host = os.getenv('MONGOHOST')
port = int(os.getenv('MONGOPORT'))
database = os.getenv('MONGODATABASE')
collection = os.getenv('MONGOCOLLECTION')

client = MongoClient(host, port)
db = client[database]
images = db[collection]

def remplir_mongo(csv : str):
    """Remplit la BDD locale MongoDB avec les images du fichier CSV issu du webscraping."""
    df = pd.read_csv(csv)
    for i in range(len(df)):
        dico = {
            f"image_id" : int(df['image_id'].loc[i]),
            f"image" : str(df['image'].loc[i]),
            }
        x = images.insert_one(dico)
    print(len(df),"Images insérées dans MongoDB")

remplir_mongo("collection_images.csv")

<h1>Lire les images en base 64</h1>

<h2>Depuis le notebook</h2>

In [ ]:
import base64
from io import BytesIO
from PIL import Image

image_base64 = df_images.loc[0, 'images']
image_bytes = base64.b64decode(image_base64)
image = Image.open(BytesIO(image_bytes))
image.show()

<h2>Depuis MongoDB</h2>

In [13]:
import os
import base64
from pymongo import MongoClient
from PIL import Image
from io import BytesIO

host = os.getenv('MONGOHOST')
port = int(os.getenv('MONGOPORT'))
database = os.getenv('MONGODATABASE')
collection = os.getenv('MONGOCOLLECTION')

client = MongoClient(host, port)
db = client[database]
images = db[collection]


document = images.find_one({"image_id": 44})

if document:
    image_base64 = document["image"]  
    image_data = base64.b64decode(image_base64)
    image = Image.open(BytesIO(image_data))
    image.show()
else:
    print("Image non trouvée !")


<h1>Script de nettoyage et d'agrégation des données</h1>

In [35]:
# Ce script gère le nettoyage et l'agrégation des données, y compris pour la première itération.

import mysql.connector
import os
from dotenv import load_dotenv
from datetime import datetime
import csv
import pandas as pd
from unidecode import unidecode
import re
from deep_translator import GoogleTranslator
from tqdm import tqdm
import sys

print("\nDébut de l'exécution du script de nettoyage :",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

load_dotenv()

#####################################################################################################################################
##### MYSQL : RECUPERER LES DERNIERS ID ET LES VARIABLES DEJA PRESSENTES DANS LA BDD POUR GERER LES INSERTIONS ET LES DOUBLONS ######
#####################################################################################################################################
#####################################################################################################################################

# user = os.getenv('USER') Je n'arrive pas pour l'instant à gérer un conflit entre user = pierre pour WSL et user = root(ce qui est attendu)
user = 'root'
# pour les autres variables cachées ça marche, WSL va bien les chercher
password = os.getenv('PASSWORD')
host = os.getenv('HOST')
database = os.getenv('DATABASE')

connexion = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database
)
curseur = connexion.cursor()

query_1 = "SELECT MAX(livre_id) FROM livres;"
curseur.execute(query_1)
result = curseur.fetchone() 
current_livre_id = (result[0] or 0)  # Pour python 5 or 0 = 5. Ca permet de gérer le None de primo nettoyage

query_2 = "SELECT MAX(auteur_id) FROM auteurs;"
curseur.execute(query_2)
result = curseur.fetchone() 
current_auteur_id = (result[0] or 0)

query_3 = "SELECT MAX(editeur_id) FROM editeurs;"
curseur.execute(query_3)
result = curseur.fetchone() 
current_editeur_id = (result[0] or 0)

query_4 = "SELECT MAX(thematique_id) FROM thematiques;"
curseur.execute(query_4)
result = curseur.fetchone() 
current_thematique_id = (result[0] or 0)

livres_deja_dans_la_bdd  =[]
query_a = "select livre_nom from livres;"
curseur.execute(query_a)
for i in curseur:
    livres_deja_dans_la_bdd.append(i[0])

auteurs_deja_dans_la_bdd  =[]
query_b = "select auteur_nom from auteurs;"
curseur.execute(query_b)
for i in curseur:
    if i[0] != None:
        auteurs_deja_dans_la_bdd.append(i[0])

editeurs_deja_dans_la_bdd  =[]
query_c = "select editeur_nom from editeurs;"
curseur.execute(query_c)
for i in curseur:
    if i[0] != None:
        editeurs_deja_dans_la_bdd.append(i[0])

thematiques_deja_dans_la_bdd  =[]
query_d = "select thematique_nom from thematiques;"
curseur.execute(query_d)
for i in curseur:
    if i[0] != None:
        thematiques_deja_dans_la_bdd.append(i[0])

dico_auteurs_deja_presents = []
query_00 = "select * from auteurs;"
curseur.execute(query_00)
for i in curseur:
    dico_auteurs_deja_presents.append(i)
dico_auteurs_deja_presents = dict(dico_auteurs_deja_presents)

dico_editeurs_deja_presents = []
query_01 = "select * from editeurs;"
curseur.execute(query_01)
for i in curseur:
    dico_editeurs_deja_presents.append(i)
dico_editeurs_deja_presents = dict(dico_editeurs_deja_presents)

dico_themes_deja_presents = []
query_02 = "select * from thematiques;"
curseur.execute(query_02)
for i in curseur:
    dico_themes_deja_presents.append(i)
dico_themes_deja_presents = dict(dico_themes_deja_presents)

curseur.close()
connexion.close()

#####################################################################################################################################
##################################################### DEBUT DU NETTOYAGE ############################################################
#####################################################################################################################################
#####################################################################################################################################

df = pd.read_csv("archives_csv/df_data_webscrap_a_nettoyer_2025-02-19_14_09_11.csv")

# Premier dataframe. Il enverra ses colonnes dans des dataframes secondaires, qui auront vocation à devenir spécifiquement chacun des
# 8 CSV d'insertion pour les 7 tables de la BDD MySQL + 1 collection d'images pour la BDD MongoDB.

df["livre_nom"] = df["livre_nom"].apply(
    lambda x : unidecode(x)
    )
df["livre_nom"] = df["livre_nom"].apply(
    lambda x : x[:255]
    )
df["livre_nom"] = df["livre_nom"].apply(
    lambda x : x.lower()
    )
df = df.drop_duplicates(subset=['livre_nom'])
df = df.drop(df[df['livre_nom'].isin(['mein kampf','my struggle'])].index)
df = df.reset_index(drop = True)

# C'est ici que l'ID est généré, après le drop duplicate par titre. Il reprend à n+1 avec n = current_livre_id
df["id"] = df.index + current_livre_id + 1

df["livre_note"] = df["livre_note"].apply(
    lambda x : re.sub(r"\(.*?\)", "", x)
    )

df["livre_date"] = df["livre_date"].apply(
    lambda x : re.findall(r'\d{4}', x)
    )

df["livre_date"] = df["livre_date"].apply(lambda x: x[0] if x else 0)
df["livre_date"] = df["livre_date"].fillna(0).astype(int)
df.loc[~df["livre_date"].between(1901, 2100), "livre_date"] = None #le format YEAR SQL considère valide la plage entre 1901 et 20155
df["livre_date"] = pd.to_datetime(df["livre_date"], format="%Y").dt.year
df["livre_date"] = df["livre_date"].replace({pd.NA: None, float("nan"): None})

df["livre_id"] = df["id"]

df["auteur_nom"] = df["auteur_nom"].apply(
    lambda x : unidecode(x).strip()
    )

df["editeur_nom"] = df["editeur_nom"].apply(
    lambda x : unidecode(x)
    )

df["thematique_nom"] = df["thematique_nom"].apply(
    lambda x : re.sub(r"[\[\]]", "", x)  
)
df["thematique_nom"] = df["thematique_nom"].apply(
    lambda x : x.split(",")
)
pd.set_option('future.no_silent_downcasting', True)
df.replace("indisponible", None, inplace=True)
df.replace('ISBN pas encore vérifié', 0, inplace=True)

nombre_de_livres_au_début = len(df)
# gestion des doublons par titre avec un masque booleen
masque = df["livre_nom"].isin(livres_deja_dans_la_bdd)
# Supprimer les lignes où le masque est True
df = df[~masque]
# A partir de là je sais que tous les "livre_id" resteront jusqu'à la fin du script.
# Je conclus donc que les tables de jonction comporteront, premières ou pas, tous ces "livre_id".
# Ce n'est pas le cas des 
# auteurs/editeurs/thematiques
# et
# auteurs/editeurs/thematiques_id
# qui pour certains seront des doublons de ce que j'avais déjà dans la BDD : ils seront droppés avant la fin du script.

if not df.empty:
    print("Le dernier webscraping apporte",len(df),"nouveaux livres. (sur les",nombre_de_livres_au_début,"qui ont été extraits.")
else:
    print("Tous les livres du dernier webscraping (",nombre_de_livres_au_début,") sont déjà connus dans la base de données. Aucun csv ne sera généré.")
    sys.exit()

#####################################################################################################################################
################################################ AGREGATION : UN DATAFRAME PAR TABLE ################################################
#####################################################################################################################################
#####################################################################################################################################

###########################################
################ df_livres ################
###########################################
df_livres = pd.DataFrame()
df_livres["livre_id"] = df["livre_id"]
df_livres["livre_nom"] = df["livre_nom"]
df_livres["livre_nombre_de_pages"] = df["livre_nombre_de_pages"]
df_livres["livre_note"] = df["livre_note"]
df_livres["ISBN_10"] = df["ISBN_10"]
df_livres["ISBN_13"] = df["ISBN_13"]
df_livres["ISBN_defaut"] = df["ISBN_defaut"]
df_livres["ISBN_verifie"] = df["ISBN_verifie"]
df_livres["livre_date"] = df["livre_date"]

###########################################
##### df_auteurs et df_auteurs_livres #####
###########################################
def assigne_un_auteur_id(auteur):
    """Pour un auteur, assigne l'id en interrogeant le dictionnaire auteurs:id de la base des données ou en génère un nouveau si besoin"""
    global current_auteur_id  # Utilisation de la variable globale current_id = je vais la mettre à jour en dehors du scope de la fonction
    for auteur_id, auteur_nom in dico_auteurs_deja_presents.items(): # .items() déstructure le dictionnaire en donnant une vue sur des tuples
        if auteur == None:
            return 1
        if auteur_nom == auteur:
            return auteur_id
    current_auteur_id += 1   
    dico_auteurs_deja_presents[current_auteur_id] = auteur
    return current_auteur_id

df["auteur_id"] = df["auteur_nom"].apply(assigne_un_auteur_id)

df_auteurs = pd.DataFrame()
df_auteurs["auteur_nom"] = df["auteur_nom"]
df_auteurs["auteur_id"] = df["auteur_id"]

df_auteurs = df_auteurs.drop_duplicates(subset=["auteur_nom"])
dico_auteurs = df_auteurs.set_index("auteur_nom")["auteur_id"].to_dict()

df_auteurs_livres = pd.DataFrame() # c'est ma table de jonction

df_auteurs_livres["livre_id"] = df["livre_id"]
df_auteurs_livres["auteur_id"] = df["auteur_nom"] # j'ai pu me permettre de faire un drop duplicates sur df_auteurs car j'avais conservé la ligne dans ce df originel
df_auteurs_livres["auteur_id"] = df_auteurs_livres["auteur_id"].map(dico_auteurs)
# map remplace chaque valeur de "auteur_id" par son ID en cherchant dans dico_auteurs
# ! Si aucun nom ne correspond dans dico_auteurs, la valeur sera remplacée par NaN 
df_auteurs_livres = df_auteurs_livres.dropna()

# je peux maintenant finir df_auteurs en supprimant ce qui sera un doublon lors de l'insertion SQL

masque_auteurs = df_auteurs["auteur_nom"].isin(auteurs_deja_dans_la_bdd)
df_auteurs = df_auteurs[~masque_auteurs] # supprimer les lignes où le masque est True 
df_auteurs = df_auteurs.dropna(subset=["auteur_nom"])

#############################################
##### df_editeurs et df_editeurs_livres #####
#############################################
def assigne_un_editeur_id(editeur):
    """Pour un editeur, assigne l'id en interrogeant le dictionnaire editeurs:id de la base des données ou en génère un nouveau si besoin"""
    global current_editeur_id  
    for editeur_id, editeur_nom in dico_editeurs_deja_presents.items(): 
        if editeur == None:
            return 1
        if editeur_nom == editeur:
            return editeur_id
    current_editeur_id += 1 
    dico_editeurs_deja_presents[current_editeur_id] = editeur
    return current_editeur_id

df_editeurs = pd.DataFrame()
df_editeurs["editeur_id"] = df["editeur_nom"].apply(assigne_un_editeur_id)
df_editeurs["editeur_nom"] = df["editeur_nom"]
df_editeurs["conservation_temporaire_de_livre_id"]= df["livre_id"]

df_editeurs_livres = pd.DataFrame()
df_editeurs_livres["editeur_id"] = df_editeurs["editeur_id"]
df_editeurs_livres["livre_id"] = df_editeurs["conservation_temporaire_de_livre_id"]
df_editeurs_livres = df_editeurs_livres.dropna()
df_editeurs_livres.drop_duplicates()

masque_editeurs = df_editeurs["editeur_nom"].isin(editeurs_deja_dans_la_bdd)

df_editeurs = df_editeurs[~masque_editeurs]
df_editeurs = df_editeurs.drop(columns="conservation_temporaire_de_livre_id")
df_editeurs = df_editeurs.drop_duplicates(subset=["editeur_nom"])
df_editeurs = df_editeurs.dropna()

###################################################
##### df_thematiques et df_thematiques_livres #####
###################################################
def assigne_un_thematique_id(thematique):
    """Pour une thematique, assigne l'id en interrogeant le dictionnaire thematiques:id de la base des données ou en génère un nouveau si besoin"""
    global current_thematique_id  
    for thematique_id, thematique_nom in dico_themes_deja_presents.items(): 
        if thematique == None:
            return 1
        if thematique_nom == thematique:
            return thematique_id
    current_thematique_id += 1 
    dico_themes_deja_presents[current_thematique_id] = thematique
    return current_thematique_id

def translate_tag_to_french(tag):
    """Traduit un tag/thématique. Détecte la langue d'origine et retourne la traduction en français."""
    traduction = GoogleTranslator(source='auto', target='fr').translate(tag)
    return traduction


future_colonne = []
for i in range(len(df)):
    ligne = []
    x = int(df["livre_id"].iloc[i])
    for j in df["thematique_nom"].iloc[i][:10]:
        tuples = (x,j)
        ligne.append(tuples)
    future_colonne.append(ligne)

df["thematique_tuples"] = future_colonne
tqdm.pandas(desc="Traduction des tags")
def traite_la_liste_de_tags(x):
    liste=[]
    for i in x:
        tag = i[1]
        tag = unidecode(tag.lower())
        tag = translate_tag_to_french(tag)
        tag = re.sub("[éèêë]","e",tag)
        tag = re.sub("[âà]","a",tag)
        tag = re.sub("[ûü]","u",tag)
        tag = re.sub("[ôö]","o",tag)
        tag = re.sub("[^a-zA-Z çïœŒ']","",tag)
        tag = re.sub("^'|'$", "", tag)
        tag = re.sub("^'|'$", "", tag)
        tag = tag.lower()
        tag = tag.lstrip()
        tag = tag.rstrip()
        if "abus" in tag:
            tag = "criminalite"
        tag = i[0],tag
        liste.append(tag)
    liste = list(set(liste))
    return liste

df["thematique_tuples"] = df["thematique_tuples"].progress_apply(traite_la_liste_de_tags)

df_thematiques = pd.DataFrame()
thematiques_du_dataset = []
conservation_des_livre_id = []
for i in range(len(df)):
    x = df["thematique_tuples"].iloc[i]
    for y in x:
        thematiques_du_dataset.append(y[1])
        conservation_des_livre_id.append(y[0])

df_thematiques["thematique_nom"] = pd.DataFrame(thematiques_du_dataset)
df_thematiques["conservation_temporaire_de_livre_id"] = pd.DataFrame(conservation_des_livre_id)
df_thematiques = df_thematiques.sort_values(by="thematique_nom")
df_thematiques = df_thematiques.reset_index(drop = True)
df_thematiques["thematique_id"] = df_thematiques["thematique_nom"].apply(assigne_un_thematique_id)
dico_thematiques = df_thematiques.set_index("thematique_nom")["thematique_id"].to_dict()

df_thematiques_livres = pd.DataFrame()
df_thematiques_livres["livre_id"] = df_thematiques["conservation_temporaire_de_livre_id"]
df_thematiques_livres["thematique_id"] = df_thematiques["thematique_id"]
df_thematiques_livres = df_thematiques_livres.dropna()
df_thematiques_livres = df_thematiques_livres.drop_duplicates()

df_thematiques = df_thematiques.drop_duplicates(subset="thematique_nom")
df_thematiques = df_thematiques.drop(columns="conservation_temporaire_de_livre_id")

masque_thematiques = df_thematiques["thematique_nom"].isin(thematiques_deja_dans_la_bdd)
df_thematiques = df_thematiques[~masque_thematiques] # supprimer les lignes où le masque est True 
df_thematiques = df_thematiques.dropna(subset=["thematique_nom"]) # préventif

#################################
########### df_images ###########
#################################
df_images = pd.DataFrame()
df_images["image"] = df["images_openlibrary"]
df_images["image_id"] = df["id"]
df_images = df_images.fillna("indisponible")

#####################################################################################################################################
############################################### GENERATION DES CSV POUR CHAQUE TABLE ################################################
#####################################################################################################################################
#####################################################################################################################################

#csv pour insertion automatisée
df_livres.to_csv('table_livres.csv', index=False)
df_auteurs.to_csv('table_auteurs.csv', index=False)
df_editeurs.to_csv('table_editeurs.csv', index=False)
df_thematiques.to_csv('table_thematiques.csv', index=False)
df_auteurs_livres.to_csv('table_auteurs_livres.csv', index=False)
df_thematiques_livres.to_csv('table_thematiques_livres.csv', index=False)
df_editeurs_livres.to_csv('table_editeurs_livres.csv', index=False)
df_images.to_csv('collection_images.csv', index=False)

#csv d'insertion pour archive
a = {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
a = re.sub("['{}]","",re.sub("[-: ]","_",str(a)))

df_livres.to_csv(f'archives_csv/table_livres{a}.csv', index=False)
df_auteurs.to_csv(f'archives_csv/table_auteurs{a}.csv', index=False)
df_editeurs.to_csv(f'archives_csv/table_editeurs{a}.csv', index=False)
df_thematiques.to_csv(f'archives_csv/table_thematiques{a}.csv', index=False)
df_auteurs_livres.to_csv(f'archives_csv/table_auteurs_livres{a}.csv', index=False)
df_thematiques_livres.to_csv(f'archives_csv/table_thematiques_livres{a}.csv', index=False)
df_editeurs_livres.to_csv(f'archives_csv/table_editeurs_livres{a}.csv', index=False)
df_images.to_csv(f'archives_csv/collection_images{a}.csv', index=False)

print("Les fichier csv pour les insertions dans les tables ont bien été générés.")
print("\nFin de l'exécution du script de nettoyage :",datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


Début de l'exécution du script de nettoyage : 2025-03-02 13:21:03
Tous les livres du dernier webscraping ( 181 ) sont déjà connus dans la base de données. Aucun csv ne sera généré.


SystemExit: 

C:\Users\Utilisateur\anaconda3\envs\projet_bloc_1\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


<h1>Tests pour l'intégrité référentielle de la BDD MySQL</h1>

J'ai dupliqué ma base de données dans sqlworkbench avec server data export/import (fichier dump)<br>
Cela, afin de faire des tests avec DELETE sans risquer des suppressions accidentelles en casade dans ma base de données principale.

Dans database = "test_db"<br><br>
Avant les DELETE, j'ai :<br><br>
SELECT * FROM livres WHERE livre_id = 1500;<br>
('288.0', 'strategic brand management', None, 1500, None, '9782857901198', 2007, 0, '9780199260003')

SELECT * FROM auteurs_livres WHERE livre_id = 1500;<br>
(1500, 1139)

SELECT * FROM auteurs WHERE auteur_id = 1139;<br>
(1139, 'Richard Elliott')

In [17]:
import os
import mysql.connector
from dotenv import load_dotenv

load_dotenv()
user = os.getenv('USER')
password = os.getenv('PASSWORD')
host = os.getenv('HOST')
database = "test_db"
try:
    mydb = mysql.connector.connect(
        user=user,
        password=password,
        host=host,
        database=database
    )
    cursor = mydb.cursor()
    print("Connexion à la base de données réussie !")
except mysql.connector.Error as err:
    print(f"Erreur de connexion : {err}")

cursor.execute("""
delete from livres where livre_id = 1500;
""")
for i in cursor:
     print(i)

Connexion à la base de données réussie !


In [18]:
cursor.execute("""
select * from livres where livre_id = 1500;
""")
for i in cursor:
     print(i)

La suppression du livre a bien fonctionné

In [22]:
cursor.execute("""
select * from auteurs_livres where livre_id = 1500;
""")
for i in cursor:
     print(i)

La suppression en cascade sur la table de jonction a bien fonctionné.

In [23]:
cursor.execute("""
select * from auteurs where auteur_id = 1139;
""")
for i in cursor:
     print(i)

(1139, 'Richard Elliott')


La suppression s'est bien arrêtée à l'auteur. Il est conservé dans la table au cas où il est lié à un autre livre.

<h1>FAST API : tests</h1>

In [2]:
# Lancer préalablement l'api depuis le terminal
import requests
# Obtenir le token
BASE_URL = "http://127.0.0.1:8000" 
token_request = {"password": "librairie"}
response = requests.post(f"{BASE_URL}/token", json=token_request)

if response.status_code == 200:
    token = response.json()["token"]
    print("Token obtenu avec succès !")
else:
    print("Erreur lors de la récupération du token :", response.json())
    token = None

# Faire une requête à l'API avec le token
if token:
    headers = {"Authorization": f"Bearer {token}"}
    params = {"limit": 10}

    response = requests.get(f"{BASE_URL}/livres", headers=headers, params=params)

    if response.status_code == 200:
        print("Données récupérées :", response.json())
    else:
        print("Erreur lors de la récupération des données :", response.json())


Token obtenu avec succès !
Données récupérées : [{'livre_nombre_de_pages': '390.0', 'livre_nom': 'laut bercerita', 'livre_note': 4.1, 'livre_id': 1, 'ISBN_10': None, 'ISBN_13': '9782895123552', 'livre_date': 2017, 'ISBN_verifie': 0, 'ISBN_defaut': '9786024246945'}, {'livre_nombre_de_pages': '384.0', 'livre_nom': 'the secret keeper of jaipur', 'livre_note': 5.0, 'livre_id': 2, 'ISBN_10': '23812252', 'ISBN_13': None, 'livre_date': 2021, 'ISBN_verifie': 0, 'ISBN_defaut': '9780778331858'}, {'livre_nombre_de_pages': '215.0', 'livre_nom': 'the world of premchand', 'livre_note': None, 'livre_id': 3, 'ISBN_10': None, 'ISBN_13': '2021443582', 'livre_date': 1969, 'ISBN_verifie': 0, 'ISBN_defaut': None}, {'livre_nombre_de_pages': '254.0', 'livre_nom': 'mansarovar - 5', 'livre_note': None, 'livre_id': 4, 'ISBN_10': None, 'ISBN_13': None, 'livre_date': 2021, 'ISBN_verifie': 0, 'ISBN_defaut': '9789390852857'}, {'livre_nombre_de_pages': '274.0', 'livre_nom': 'mansarovar - part 6', 'livre_note': None,

<h1>Notes</h1>

J'ai rencontré cette difficulté avec crontab :  
Sous WSL, crontab ne se relançait jamais au redémarrage de l'ordinateur.  
Il fallait à chaque fois:  
-ouvrir le terminal wsl  
-lancer la commande sudo service cron start dans le terminal

... ou alors ne jamais éteindre l'ordinateur !!!

Solution qui permet de rester avec windows 10 et wsl 1   :  
Les tâches au démarrage se trouvent dans le fichier .bashrc  
Donc avec  
nano ~/.bashrc  
j'ai rajouté une instruction pour chaque redémarrage du terminal, le fameux :  
sudo service cron start  
j'ai par ailleurs désactivé la demande de mot de passe en modifiant le fichier dédié "sudoers" via visudo qui est l'éditeur recommandé :  
sudo visudo  
Ca nous ouvre donc l'édition du fichier sudoers à la fin duquel on rajoute:  
utilisateur(mets ton prénom) ALL=(ALL) NOPASSWD: /usr/sbin/service cron start

Depuis, à l'ouverture du terminal ubuntu, la première chose qui s'écrit est :  
Starting periodic command scheduler cron     OK  
 
Il ne reste plus qu'à automatiser l'ouverture d'ubuntu via windows à chaque redémarrage de l'ordinateur  :

je mets le wsl.exe de "C:\Windows\System32"  
dans  
"C:\Users\Utilisateur\AppData\Roaming\Microsoft\Windows\Start Menu\Programs\Startup"

Et ça marche !

https://www.mediamarkt.ch/fr/content/ordinateurs-bureau/software-apps/windows-10-demarrage-automatique-suppression-de-programmes#Comment%20supprimer%20des%20programmes%20du%20d%C3%A9marrage%20automatique%20de%20Windows%2010